In [3]:
# Import Step
# Template + Guide to use SQL in Google Colab/Jupytyr Notebook
import pandas as pd
import sqlite3

I am using a tried and true method I use for using SQL within a Juptyr notebook. I like this method because I can quickly import a .csv or an .xls file while I'm on the go. I already have this saved to my drive. It is a template basically.

In [4]:
# Setup Step
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In a real world seetting. I would use the company "preferred" IDE. At RTI I usually use Visual Studio for Python, for SQL we usually use a web-browser integrated server. 

In [5]:
# File Upload
input_df = pd.read_csv('/content/sample_claims.csv')

# After first time, you do this. If you try to reload the code, you'll get the "Table Exists" error code
# Setting the table, first time step
pd_to_sqlDB(input_df,
            table_name='sample_claims',
           db_name='default.db')

# Show first ten rows of dataset
sql_query_string = """
SELECT *
FROM sample_claims
LIMIT 10
"""

# String to run the query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

2022-04-08 01:49:52 INFO: SQL DB default.db created
2022-04-08 01:49:52 INFO: SQL Table sample_claims created with 6 columns
2022-04-08 01:49:52 INFO: 5000 rows uploaded to sample_claims


,patient_id,claim_id,diagnosis_codes,procedure_code,date_service,date_received
0,A1670,1.0,Z01.419^Z11.51,99999,2021-01-25,2021-01-26
1,A0086,2.0,Z01.419^Z12.4,99999,2021-01-27,2021-01-29
2,A0086,3.0,Z12.4^Z11.51,87491,2021-01-07,2021-01-10
3,A0086,4.0,R30.0,86735,2021-01-15,2021-01-16
4,A0311,5.0,Z34.81^I10^G40.909^E66.9,83014,2021-01-06,2021-01-07
5,A0311,6.0,None,85049,2021-01-08,2021-01-09
6,A0311,7.0,N92.6^Z11.3^Z11.8^Z32.01,84132,2021-01-07,2021-01-08
7,A0311,8.0,Z01.419^Z12.4,83615,2021-01-14,2021-01-15
8,A0311,9.0,M25.559^M48.00^M54.16^R76.9,86696,2021-01-12,2021-01-16
9,A0311,10.0,N76.0,86900,2021-01-07,2021-01-09


Hooray it worked! Imported the data and ran a quick script to make sure the table shows up.

In [6]:
#1a - Determine the 5 most common valid diagnosis codes
sql_query_string = """
SELECT diagnosis_codes,
COUNT(diagnosis_codes) AS diagnosis_count
FROM sample_claims
GROUP BY diagnosis_codes
ORDER BY diagnosis_count DESC, diagnosis_codes DESC
LIMIT 5
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,diagnosis_codes,diagnosis_count
0,N92.6^Z11.3^Z11.8^Z32.01,152
1,Z34.81,128
2,Z00.00,107
3,N76.0,86
4,N92.6^Z11.3^Z11.8,81


In [7]:
#1b - How many patients are associated with at least one of those diagnoses?

sql_query_string = """
SELECT diagnosis_codes, COUNT(DISTINCT patient_id) AS count
FROM sample_claims
WHERE diagnosis_codes IN ("Z00.00","N76.0","Z34.81", "N92.6^Z11.3^Z11.8", "N92.6^Z11.3^Z11.8^Z32.01")
GROUP BY diagnosis_codes
ORDER BY diagnosis_codes DESC
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,diagnosis_codes,count
0,Z34.81,38
1,Z00.00,31
2,N92.6^Z11.3^Z11.8^Z32.01,31
3,N92.6^Z11.3^Z11.8,29
4,N76.0,29
